In [2]:
import pandas as pd

# Load the datasets (adjust the paths as necessary)
amazon_data = pd.read_csv(r"C:\Users\Abinaya\OneDrive\Desktop\Netflix-Prime-Hotstar-Dashboard-Power-BI-main\dataset\amazon_prime_titles.csv")
netflix_data = pd.read_csv(r"C:\Users\Abinaya\OneDrive\Desktop\Netflix-Prime-Hotstar-Dashboard-Power-BI-main\dataset\netflix_titles.csv")
hotstar_data = pd.read_csv(r"C:\Users\Abinaya\OneDrive\Desktop\Netflix-Prime-Hotstar-Dashboard-Power-BI-main\dataset\disney_plus_titles.csv")


# Check for missing values in each column
print(amazon_data.isnull().sum())


show_id            0
type               0
title              0
director        2083
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64


In [3]:
# 1. Fill missing 'director' with 'Unknown'
amazon_data['director'] = amazon_data['director'].fillna('Unknown')

# 2. Fill missing 'cast' with 'Unknown'
amazon_data['cast'] = amazon_data['cast'].fillna('Unknown')

# 3. Fill missing 'country' with 'Unknown'
amazon_data['country'] = amazon_data['country'].fillna('Unknown')

# 4. Fill missing 'date_added' with the most common date (mode)
amazon_data_date_added = amazon_data['date_added'].mode()[0]
amazon_data['date_added'] = amazon_data['date_added'].fillna(amazon_data_date_added)

# 5. Fill missing 'rating' with the most common rating
amazon_rating = amazon_data['rating'].mode()[0]
amazon_data['rating'] = amazon_data['rating'].fillna(amazon_rating)



# Print statements to verify that the changes have been applied correctly
print("Filled missing 'director' values: ", amazon_data['director'].isnull().sum())
print("Filled missing 'cast' values: ", amazon_data['cast'].isnull().sum())
print("Filled missing 'country' values: ", amazon_data['country'].isnull().sum())
print("Filled missing 'date_added' values: ", amazon_data['date_added'].isnull().sum())
print("Filled missing 'rating' values: ", amazon_data['rating'].isnull().sum())


Filled missing 'director' values:  0
Filled missing 'cast' values:  0
Filled missing 'country' values:  0
Filled missing 'date_added' values:  0
Filled missing 'rating' values:  0


In [4]:
import pandas as pd
import numpy as np

# Function to convert duration to minutes
def convert_duration_to_minutes(duration):
    if isinstance(duration, float) and np.isnan(duration):
        # Handle NaN values (leave them as NaN for now)
        return np.nan
    elif 'Season' in duration:
        # Extract the number of seasons and multiply by 125
        seasons = int(duration.split(' ')[0])
        return seasons * 125
    else:
        # Extract the numeric part of the minutes
        return int(duration.split(' ')[0])

# Apply the function to the duration column
amazon_data['duration_minutes'] = amazon_data['duration'].apply(convert_duration_to_minutes)

# # Drop rows where 'duration_minutes' is NaN
# combined_data.dropna(subset=['duration_minutes'], inplace=True)

# Verify the result
print(amazon_data[['duration', 'duration_minutes']].head())

# Drop the original 'duration' column if necessary
amazon_data.drop('duration', axis=1, inplace=True)

  duration  duration_minutes
0  113 min               113
1  110 min               110
2   74 min                74
3   69 min                69
4   45 min                45


In [5]:
print(amazon_data.isnull().sum())

show_id             0
type                0
title               0
director            0
cast                0
country             0
date_added          0
release_year        0
rating              0
listed_in           0
description         0
duration_minutes    0
dtype: int64


In [6]:
def create_churn_column(data):
    # Define churn criteria
    churn_condition = data['rating'].isin(['R', '18+', 'NR', '16+'])  # Add more criteria as needed
    data['churn'] = np.where(churn_condition, 1, 0)  # 1 for churn, 0 for no churn
    return data

# Apply the function to the Amazon data
amazon_data = create_churn_column(amazon_data)

In [7]:
print(amazon_data.tail())

     show_id     type                    title         director  \
9663   s9664    Movie      Pride Of The Bowery  Joseph H. Lewis   
9664   s9665  TV Show            Planet Patrol          Unknown   
9665   s9666    Movie                  Outpost     Steve Barker   
9666   s9667  TV Show  Maradona: Blessed Dream          Unknown   
9667   s9668    Movie              Harry Brown    Daniel Barber   

                                                   cast  country  \
9663                           Leo Gorcey, Bobby Jordan  Unknown   
9664  DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...  Unknown   
9665  Ray Stevenson, Julian Wadham, Richard Brake, M...  Unknown   
9666  Esteban Recagno, Ezequiel Stremiz, Luciano Vit...  Unknown   
9667  Michael Caine, Emily Mortimer, Joseph Gilgun, ...  Unknown   

          date_added  release_year rating                listed_in  \
9663  March 30, 2021          1940     7+                   Comedy   
9664  March 30, 2021          2018    13+       

In [8]:
output_file_path = r"C:\Users\Abinaya\OneDrive\Documents\ABI\BI Lab\Netflix-Prime-Hotstar-Dashboard-Power-BI-main\dataset\updated_amazon_data.xlsx"
amazon_data.to_excel(output_file_path, index=False)

print("Churn column saved to Excel file successfully.")


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Abinaya\\OneDrive\\Documents\\ABI\\BI Lab\\Netflix-Prime-Hotstar-Dashboard-Power-BI-main\\dataset\\updated_amazon_data.xlsx'

In [43]:
print(amazon_data['churn'].value_counts())

churn
0    5645
1    4023
Name: count, dtype: int64


In [25]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [44]:
features = amazon_data[['rating', 'duration_minutes', 'release_year', 'country']]  # You can add more columns if needed
features = pd.get_dummies(features, columns=['rating', 'country'], drop_first=True)
labels = amazon_data['churn']

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Initialize and train the SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)


# Predict on the test data
y_pred = svm_model.predict(X_test)

# Print confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[1079    0]
 [ 855    0]]
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      1079
           1       0.00      0.00      0.00       855

    accuracy                           0.56      1934
   macro avg       0.28      0.50      0.36      1934
weighted avg       0.31      0.56      0.40      1934



C:\Users\Abinaya\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abinaya\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abinaya\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [46]:
import joblib
joblib.dump(svm_model, 'amazon_svm_model.pkl')

['amazon_svm_model.pkl']

In [47]:
svm_model = joblib.load('amazon_svm_model.pkl')

In [48]:
y_pred = svm_model.predict(X_test)

# Print the confusion matrix and classification report
from sklearn.metrics import classification_report, confusion_matrix

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification report (provides precision, recall, f1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[1079    0]
 [ 855    0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      1.00      0.72      1079
           1       0.00      0.00      0.00       855

    accuracy                           0.56      1934
   macro avg       0.28      0.50      0.36      1934
weighted avg       0.31      0.56      0.40      1934



C:\Users\Abinaya\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abinaya\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abinaya\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [49]:
svm_model = SVC(class_weight='balanced')

# Fit the model again
svm_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = svm_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[  60 1019]
 [  12  843]]
              precision    recall  f1-score   support

           0       0.83      0.06      0.10      1079
           1       0.45      0.99      0.62       855

    accuracy                           0.47      1934
   macro avg       0.64      0.52      0.36      1934
weighted avg       0.67      0.47      0.33      1934



In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
rf_model = RandomForestClassifier(random_state=42)
features = amazon_data[['rating', 'duration_minutes']]  # Add more relevant features if needed
labels = amazon_data['churn']

# Convert categorical variables into dummy/indicator variables
features = pd.get_dummies(features, columns=['rating'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Fit the model on the training data
rf_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred_rf = rf_model.predict(X_test)

# Print the confusion matrix

print("Random Forest Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

# Print the classification report
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

Random Forest Confusion Matrix:
[[1079    0]
 [   0  855]]
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1079
           1       1.00      1.00      1.00       855

    accuracy                           1.00      1934
   macro avg       1.00      1.00      1.00      1934
weighted avg       1.00      1.00      1.00      1934



In [14]:

# Save the model to a file
joblib.dump(rf_model, 'amazon_rf_model.pkl')
print("Random Forest model saved successfully.")

Random Forest model saved successfully.


In [15]:
amazon_data['predicted_churn'] = rf_model.predict(features)  # Make predictions on the original dataset

# Save the updated DataFrame with the predicted churn to a new Excel file
output_file_path = r"C:\Users\Abinaya\OneDrive\Documents\ABI\BI Lab\Netflix-Prime-Hotstar-Dashboard-Power-BI-main\dataset\updated_amazon_data_with_predictions.xlsx"
amazon_data.to_excel(output_file_path, index=False)

print("Updated DataFrame with predicted churn saved to Excel file successfully.")

Updated DataFrame with predicted churn saved to Excel file successfully.


In [17]:
original_file_path = r"C:\Users\Abinaya\OneDrive\Documents\ABI\BI Lab\Netflix-Prime-Hotstar-Dashboard-Power-BI-main\dataset\amazon_prime_titles.csv"
original_data = pd.read_excel(original_file_path)

# Predict churn using the random forest model
# Assuming 'features' contains the correct features used for prediction
original_data['predicted_churn'] = rf_model.predict(features)  # Make predictions on the original dataset

# Save the updated DataFrame back to the original Excel file
original_data.to_excel(original_file_path, index=False)

print("Updated DataFrame with predicted churn saved to the original Excel file successfully.")

ValueError: Excel file format cannot be determined, you must specify an engine manually.

SyntaxError: invalid syntax (2103225878.py, line 1)